# Simple Ising Model Example

This notebook demonstrates:
1. Creating an Ising model (from SpinSystems submodule)
2. Setting up a Metropolis algorithm with Boltzmann weights
3. Configuring measurements with the new measurement framework
4. Running equilibrium Monte Carlo simulation
5. Analyzing results

In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using Random
using StatsBase
using MonteCarloX
using SpinSystems

  Activating project at `~/.julia/dev/MonteCarloX`
Precompiling packages...
   1960.5 ms  ✓ SpinSystems
  1 dependency successfully precompiled in 4 seconds. 236 already precompiled.


UndefVarError: UndefVarError: `SpinSystems` not defined in `MonteCarloX`
Suggestion: check for spelling errors or missing imports.

## Setup System and Algorithm

In [2]:
# Setup
rng = MersenneTwister(42)
L = 8
# Create Ising system
sys = Ising([L, L], J=1, periodic=true)
init!(sys, :random, rng=rng)

println("System configuration:")
println("  Lattice size: $(L)×$(L)")
println("  Total spins: $(L*L)")
println("  Spins from system: $(size(sys.spins))")

# Create Metropolis algorithm with Boltzmann weight
β = 0.3  # Inverse temperature
alg = Metropolis(rng, β=β)
println("  Inverse temperature β: $β")

UndefVarError: UndefVarError: `Ising` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Configure Measurements

In [3]:
# Setup measurements
measurements = Measurements([
    :energy => energy => Float64[],
    :magnetization => magnetization => Float64[]
], interval=10)

UndefVarError: UndefVarError: `energy` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Run Simulation

In [4]:
# Thermalization
println("Thermalizing...")
N = L * L
for i in 1:N*1000
    spin_flip!(sys, alg)
end
println("  Thermalization complete: $(alg.steps) steps")

# Reset statistics after thermalization
reset_statistics!(alg)

Thermalizing...


UndefVarError: UndefVarError: `spin_flip!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
# Production run
println("Production run...")
for i in 1:N*10000
    spin_flip!(sys, alg)
    measure!(measurements, sys, i)
end

Production run...


UndefVarError: UndefVarError: `spin_flip!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Benchmark code

In [6]:
# benchmark the spinflip here
using BenchmarkTools
@btime spin_flip!(sys, alg)

UndefVarError: UndefVarError: `spin_flip!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Analyze Results

In [7]:
energies = measurements[:energy].data
mags = measurements[:magnetization].data

avg_E = mean(energies) / N
avg_M = mean(mags) / N

println("\nResults:")
println("  Total steps: $(alg.steps)")
println("  Acceptance rate: $(round(acceptance_rate(alg), digits=4))")
println("\nObservables (per spin):")
println("  Average energy: $(round(avg_E, digits=4))")
println("  Average magnetization: $(round(avg_M, digits=4))")

UndefVarError: UndefVarError: `measurements` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Energy Time Series

In [8]:
using Plots
energies = measurements[:energy].data
plot(energies, label="Energy Time Series", xlabel="Monte Carlo Steps", ylabel="Energy")

UndefVarError: UndefVarError: `measurements` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Energy Distribution vs Exact (Beale 8x8)

In [9]:
log_dos_beale_8x8 = [ (-128, 0.6931471805599453), (-124, 0.0), (-120, 4.852030263919617), (-116, 5.545177444479562), (-112, 8.449342524508063), (-108, 9.793672686528922), (-104, 11.887298863200714), (-100, 13.477180596840947), (-96, 15.268195474147658), (-92, 16.912371686315282), (-88, 18.59085846191256), (-84, 20.230089202801466), (-80, 21.870810400320693), (-76, 23.498562234123614), (-72, 25.114602234581373), (-68, 26.70699035290573), (-64, 28.266152815389898), (-60, 29.780704423363996), (-56, 31.241053997806176), (-52, 32.63856452513369), (-48, 33.96613536105969), (-44, 35.217576663643314), (-40, 36.3873411250109), (-36, 37.47007844691906), (-32, 38.46041522581422), (-28, 39.35282710786369), (-24, 40.141667825183845), (-20, 40.82130289691285), (-16, 41.38631975325592), (-12, 41.831753810069756), (-8, 42.153328313883975), (-4, 42.34770636939425), (0, 42.41274640460084), (4, 42.34770636939425), (8, 42.153328313883975), (12, 41.831753810069756), (16, 41.38631975325592), (20, 40.82130289691285), (24, 40.141667825183845), (28, 39.35282710786369), (32, 38.46041522581422), (36, 37.47007844691906), (40, 36.3873411250109), (44, 35.217576663643314), (48, 33.96613536105969), (52, 32.63856452513369), (56, 31.241053997806176), (60, 29.780704423363996), (64, 28.266152815389898), (68, 26.70699035290573), (72, 25.114602234581373), (76, 23.498562234123614), (80, 21.870810400320693), (84, 20.230089202801466), (88, 18.59085846191256), (92, 16.912371686315282), (96, 15.268195474147658), (100, 13.477180596840947), (104, 11.887298863200714), (108, 9.793672686528922), (112, 8.449342524508063), (116, 5.545177444479562), (120, 4.852030263919617), (124, 0.0), (128, 0.6931471805599453) ];
struct BoltzmannDistribution
    pdf::Dict{Any,Float64}
    function log_sum(a::Number, b::Number)::Float64
        if b < a
            return a + log(1 + exp(b - a))
        else
            return b + log(1 + exp(a - b))
        end
    end
    function initialize_BoltzmannDistribution(beta, log_dos)
        log_Z = -Inf
        for (E, log_d) in log_dos
            log_Z = log_sum(log_Z, log_d - beta * E)
        end
        pdf = Dict{Int64,Float64}()
        for (E, log_d) in log_dos
            log_pdf = log_d - beta * E - log_Z
            pdf[E] = exp(log_pdf)
        end
        return pdf
    end
    BoltzmannDistribution(beta, log_dos) = new(initialize_BoltzmannDistribution(beta, log_dos))
end

energies = measurements[:energy].data
hist = fit(Histogram, energies, minimum(energies)-2:4:maximum(energies)+2, closed=:left)
dist = StatsBase.normalize(hist, mode=:probability)
plot(dist, label="Energy Distribution", xlabel="Energy", ylabel="Probability",
    title="Energy Distribution of 2D Ising Model at beta=$(alg.logweight.β)")

P_true = BoltzmannDistribution(alg.logweight.β, log_dos_beale_8x8).pdf
E_vals = sort(collect(keys(P_true)))
P_vals = [P_true[E] for E in E_vals]
plot!(E_vals, P_vals, label="Exact Distribution", color=:black, lw=2)

UndefVarError: UndefVarError: `measurements` not defined in `Main`
Suggestion: check for spelling errors or missing imports.